In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import string
import os
from openai import OpenAI
import json
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Znatno_personal/Отчетность ВКР/Артефакты/Parsing_avito/my_data22.csv')

In [ ]:
df['info'] = df['О себе'].astype(str) + ' ' + 'Обязанности: ' +  df['Обязанности'].astype(str)+ ' ' + 'Компания: ' + df['Компания'].astype(str) + ' ' + 'Учебные заведения: ' + df['Учебные заведения'].astype(str) + ' ' + 'Стаж работы: ' + df['Стаж работы'].astype(str) + ' ' + 'Образование: ' + df['Образование'].astype(str)

In [ ]:
api_key = API_KEY
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
def deep_seekgpt(messages, temperature):
  response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    temperature = temperature,
    response_format={
        'type': 'json_object'
    }
   )

#reasoning_content = response.choices[0].message.reasoning_content
  content = response.choices[0].message.content
  return content

In [ ]:
data = df.copy()

In [ ]:
data1 = df[df['Оценка'] == 0].sample(n=51)
data2 = df[df['Оценка'] == 1].sample(n=51)
data3 = df[df['Оценка'] == 2].sample(n=51)
data4 = df[df['Оценка'] == 3].sample(n=51)
data5 = df[df['Оценка'] == 4].sample(n=43)
data6 = df[df['Оценка'] == 5].sample(n=51)
data7 = df[df['Оценка'] == 6].sample(n=51)
data8 = df[df['Оценка'] == 7].sample(n=51)
data9 = df[df['Оценка'] == 8].sample(n=51)

In [ ]:
data = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9])

In [ ]:
X = data['info']
y = data['Оценка']

In [ ]:
y = np.array(y)

In [ ]:
result = {}

In [ ]:
for i in X.index:
  messages = [{"role": "user", "content": f"Классифицируй резюме косметологов по 9 классам (0–8): 0 класс - это нерелеватные резюме, у кандидатов стаж работы 0 лет и очень часто встречается не указано, учебные заведения не указаны \n\
            1  - кандидаты имеют медицинское образование, но стаж работы 0 лет, 2 - Все кандидаты имеют опыт от 0.001 до 1 года и любое образование \n\
            3 - кандидаты в начале резюме указывают опыт работы n-лет, где n - любое число, но Компания: не указано, Должность: не указано, Учебное заведение: не указано \n\
            4 - Кандидаты имеют стаж работы > 2 лет, указывают компанию, должность, но Учебное заведение: не указано или не медицинское \n\
            5 - Кандидаты имеют среднее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает: должность, компанию,,учебное заведение: медицинский колледж школы обучающие косметологов \n\
            6 - Кандидаты имеют высшее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            7 - Кандидаты имеют среднее медицинское образование и стаж работы >3 лет,Кандидат указывает: должность, компанию, учебное заведение: медицинский колледж школы обучающие косметологов \n\
            8 - Кандидаты имеют высшее медицинское образование и стаж работы > 3 лет. Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            Высшее медицинское образование - это медицинские университеты и институты. Среднее медицинское образование - это медицинский колледж, школы обучающие косметологов. Стаж работы - это опыт работы в сфере косметологии. \n\
            Обрати внимание на класс 3 и 4 прежде чем принять решение.\n\ Не обращай внимание на образование, оно не влияет на метку класса \n\
            Три примера для  каждого класса:\n\
                \"text\": {df['info'][df['Оценка'] == 0].sample(n=3).to_list()}, \"label\": 0,\n\
                \"text\": {df['info'][df['Оценка'] == 1].sample(n=3).to_list()}, \"label\": 1,\n\
                \"text\": {df['info'][df['Оценка'] == 2].sample(n=3).to_list()}, \"label\": 2,\n\
                \"text\": {df['info'][df['Оценка'] == 3].sample(n=3).to_list()}, \"label\": 3,\n\
                \"text\": {df['info'][df['Оценка'] == 4].sample(n=3).to_list()}, \"label\": 4,\n\
                \"text\": {df['info'][df['Оценка'] == 5].sample(n=3).to_list()}, \"label\": 5,\n\
                \"text\": {df['info'][df['Оценка'] == 6].sample(n=3).to_list()}, \"label\": 6,\n\
                \"text\": {df['info'][df['Оценка'] == 7].sample(n=3).to_list()}, \"label\": 7,\n\
                \"text\": {df['info'][df['Оценка'] == 8].sample(n=3).to_list()}, \"label\": 8\n\
            Угадай класс данного резюме: {data['info'][i]}. Дай ответ в формате json с двумя ключами 'text' - полученный текст и prediction - угаданная метка класса" }]
  answer = deep_seekgpt(messages)
  answer = json.loads(answer)
  result[i] = answer1['prediction']

In [ ]:
pred = pd.DataFrame.from_dict(result, orient='index', columns=['prediction'])

In [ ]:
print("Accuracy :", accuracy_score(y, pred['prediction2']))
print("F1 score:",f1_score(y, pred['prediction2'], average='weighted'))

Accuracy : 0.6873614190687362
F1 score: 0.6798663647468272


#2 проход

In [ ]:
result1 = {}

In [ ]:
for i in X.index:
  messages = [{"role": "user", "content": f"Классифицируй резюме косметологов по 9 классам (0–8): 0 класс - это нерелеватные резюме, у кандидатов стаж работы 0 лет и очень часто встречается не указано, учебные заведения не указаны \n\
            1  - кандидаты имеют медицинское образование, но стаж работы 0 лет, 2 - Все кандидаты имеют опыт от 0.001 до 1 года и любое образование \n\
            3 - кандидаты в начале резюме указывают опыт работы n-лет, где n - любое число, но Компания: не указано, Должность: не указано, Учебное заведение: не указано \n\
            4 - Кандидаты имеют стаж работы > 2 лет, указывают компанию, должность, но Учебное заведение: не указано или не медицинское \n\
            5 - Кандидаты имеют среднее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает: должность, компанию,,учебное заведение: медицинский колледж школы обучающие косметологов \n\
            6 - Кандидаты имеют высшее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            7 - Кандидаты имеют среднее медицинское образование и стаж работы >3 лет,Кандидат указывает: должность, компанию, учебное заведение: медицинский колледж школы обучающие косметологов \n\
            8 - Кандидаты имеют высшее медицинское образование и стаж работы > 3 лет. Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            Высшее медицинское образование - это медицинские университеты и институты. Среднее медицинское образование - это медицинский колледж, школы обучающие косметологов. Стаж работы - это опыт работы в сфере косметологии. \n\
            Обрати внимание на класс 3 и 4 прежде чем принять решение.\n\ Не обращай внимание на образование, оно не влияет на метку класса \n\
            Три примера для  каждого класса:\n\
                \"text\": {df['info'][df['Оценка'] == 0].sample(n=3).to_list()}, \"label\": 0,\n\
                \"text\": {df['info'][df['Оценка'] == 1].sample(n=3).to_list()}, \"label\": 1,\n\
                \"text\": {df['info'][df['Оценка'] == 2].sample(n=3).to_list()}, \"label\": 2,\n\
                \"text\": {df['info'][df['Оценка'] == 3].sample(n=3).to_list()}, \"label\": 3,\n\
                \"text\": {df['info'][df['Оценка'] == 4].sample(n=3).to_list()}, \"label\": 4,\n\
                \"text\": {df['info'][df['Оценка'] == 5].sample(n=3).to_list()}, \"label\": 5,\n\
                \"text\": {df['info'][df['Оценка'] == 6].sample(n=3).to_list()}, \"label\": 6,\n\
                \"text\": {df['info'][df['Оценка'] == 7].sample(n=3).to_list()}, \"label\": 7,\n\
                \"text\": {df['info'][df['Оценка'] == 8].sample(n=3).to_list()}, \"label\": 8\n\
            Угадай класс данного резюме: {data['info'][i]}. Дай ответ в формате json с двумя ключами 'text' - полученный текст и prediction - угаданная метка класса" }]
  answer = deep_seekgpt(messages, temperature = 0.5)
  answer1 = json.loads(answer)
  result1[i] = answer1['prediction']

In [ ]:
result1 = pd.DataFrame.from_dict(result1, orient='index', columns=['prediction1'])

#проход 3

In [ ]:
result2 = {}

In [ ]:
for i in X.index:
  messages = [{"role": "user", "content": f"Классифицируй резюме косметологов по 9 классам (0–8): 0 класс - это нерелеватные резюме, у кандидатов стаж работы 0 лет и очень часто встречается не указано, учебные заведения не указаны \n\
            1  - кандидаты имеют медицинское образование, но стаж работы 0 лет, 2 - Все кандидаты имеют опыт от 0.001 до 1 года и любое образование \n\
            3 - кандидаты в начале резюме указывают опыт работы n-лет, где n - любое число, но Компания: не указано, Должность: не указано, Учебное заведение: не указано \n\
            4 - Кандидаты имеют стаж работы > 2 лет, указывают компанию, должность, но Учебное заведение: не указано или не медицинское \n\
            5 - Кандидаты имеют среднее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает: должность, компанию,,учебное заведение: медицинский колледж школы обучающие косметологов \n\
            6 - Кандидаты имеют высшее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            7 - Кандидаты имеют среднее медицинское образование и стаж работы >3 лет,Кандидат указывает: должность, компанию, учебное заведение: медицинский колледж школы обучающие косметологов \n\
            8 - Кандидаты имеют высшее медицинское образование и стаж работы > 3 лет. Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            Высшее медицинское образование - это медицинские университеты и институты. Среднее медицинское образование - это медицинский колледж, школы обучающие косметологов. Стаж работы - это опыт работы в сфере косметологии. \n\
            Обрати внимание на класс 3 и 4 прежде чем принять решение.\n\ Не обращай внимание на образование, оно не влияет на метку класса \n\
            Три примера для  каждого класса:\n\
                \"text\": {df['info'][df['Оценка'] == 0].sample(n=3).to_list()}, \"label\": 0,\n\
                \"text\": {df['info'][df['Оценка'] == 1].sample(n=3).to_list()}, \"label\": 1,\n\
                \"text\": {df['info'][df['Оценка'] == 2].sample(n=3).to_list()}, \"label\": 2,\n\
                \"text\": {df['info'][df['Оценка'] == 3].sample(n=3).to_list()}, \"label\": 3,\n\
                \"text\": {df['info'][df['Оценка'] == 4].sample(n=3).to_list()}, \"label\": 4,\n\
                \"text\": {df['info'][df['Оценка'] == 5].sample(n=3).to_list()}, \"label\": 5,\n\
                \"text\": {df['info'][df['Оценка'] == 6].sample(n=3).to_list()}, \"label\": 6,\n\
                \"text\": {df['info'][df['Оценка'] == 7].sample(n=3).to_list()}, \"label\": 7,\n\
                \"text\": {df['info'][df['Оценка'] == 8].sample(n=3).to_list()}, \"label\": 8\n\
            Угадай класс данного резюме: {data['info'][i]}. Дай ответ в формате json с двумя ключами 'text' - полученный текст и prediction - угаданная метка класса" }]
  answer =  deep_seekgpt(messages, temperature = 0.2)
  answer = json.loads(answer)
  result2[i] = answer1['prediction']

In [ ]:
result2 = pd.DataFrame.from_dict(result2, orient='index', columns=['prediction2'])

#проход 4

In [ ]:
result3 = {}

In [ ]:
for i in X.index:
  messages = [{"role": "user", "content": f"Классифицируй резюме косметологов по 9 классам (0–8): 0 класс - это нерелеватные резюме, у кандидатов стаж работы 0 лет и очень часто встречается не указано, учебные заведения не указаны \n\
            1  - кандидаты имеют медицинское образование, но стаж работы 0 лет, 2 - Все кандидаты имеют опыт от 0.001 до 1 года и любое образование \n\
            3 - кандидаты в начале резюме указывают опыт работы n-лет, где n - любое число, но Компания: не указано, Должность: не указано, Учебное заведение: не указано \n\
            4 - Кандидаты имеют стаж работы > 2 лет, указывают компанию, должность, но Учебное заведение: не указано или не медицинское \n\
            5 - Кандидаты имеют среднее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает: должность, компанию,,учебное заведение: медицинский колледж школы обучающие косметологов \n\
            6 - Кандидаты имеют высшее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            7 - Кандидаты имеют среднее медицинское образование и стаж работы >3 лет,Кандидат указывает: должность, компанию, учебное заведение: медицинский колледж школы обучающие косметологов \n\
            8 - Кандидаты имеют высшее медицинское образование и стаж работы > 3 лет. Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            Высшее медицинское образование - это медицинские университеты и институты. Среднее медицинское образование - это медицинский колледж, школы обучающие косметологов. Стаж работы - это опыт работы в сфере косметологии. \n\
            Обрати внимание на класс 3 и 4 прежде чем принять решение.\n\ Не обращай внимание на образование, оно не влияет на метку класса \n\
            Три примера для  каждого класса:\n\
                \"text\": {df['info'][df['Оценка'] == 0].sample(n=3).to_list()}, \"label\": 0,\n\
                \"text\": {df['info'][df['Оценка'] == 1].sample(n=3).to_list()}, \"label\": 1,\n\
                \"text\": {df['info'][df['Оценка'] == 2].sample(n=3).to_list()}, \"label\": 2,\n\
                \"text\": {df['info'][df['Оценка'] == 3].sample(n=3).to_list()}, \"label\": 3,\n\
                \"text\": {df['info'][df['Оценка'] == 4].sample(n=3).to_list()}, \"label\": 4,\n\
                \"text\": {df['info'][df['Оценка'] == 5].sample(n=3).to_list()}, \"label\": 5,\n\
                \"text\": {df['info'][df['Оценка'] == 6].sample(n=3).to_list()}, \"label\": 6,\n\
                \"text\": {df['info'][df['Оценка'] == 7].sample(n=3).to_list()}, \"label\": 7,\n\
                \"text\": {df['info'][df['Оценка'] == 8].sample(n=3).to_list()}, \"label\": 8\n\
            Угадай класс данного резюме: {data['info'][i]}. Дай ответ в формате json с двумя ключами 'text' - полученный текст и prediction - угаданная метка класса" }]
  answer =  deep_seekgpt(messages, temperature = 1.5)
  answer1 = json.loads(answer)
  result3[i] = answer1['prediction']

In [ ]:
result3 = pd.DataFrame.from_dict(result3, orient='index', columns=['prediction3'])

# 5 проход

In [ ]:
result4 = {}

In [ ]:
for i in X.index:
  messages = [{"role": "user", "content": f"Классифицируй резюме косметологов по 9 классам (0–8): 0 класс - это нерелеватные резюме, у кандидатов стаж работы 0 лет и очень часто встречается не указано, учебные заведения не указаны \n\
            1  - кандидаты имеют медицинское образование, но стаж работы 0 лет, 2 - Все кандидаты имеют опыт от 0.001 до 1 года и любое образование \n\
            3 - кандидаты в начале резюме указывают опыт работы n-лет, где n - любое число, но Компания: не указано, Должность: не указано, Учебное заведение: не указано \n\
            4 - Кандидаты имеют стаж работы > 2 лет, указывают компанию, должность, но Учебное заведение: не указано или не медицинское \n\
            5 - Кандидаты имеют среднее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает: должность, компанию,,учебное заведение: медицинский колледж школы обучающие косметологов \n\
            6 - Кандидаты имеют высшее медицинское образование и стаж работы  от 1 до 3 лет,Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            7 - Кандидаты имеют среднее медицинское образование и стаж работы >3 лет,Кандидат указывает: должность, компанию, учебное заведение: медицинский колледж школы обучающие косметологов \n\
            8 - Кандидаты имеют высшее медицинское образование и стаж работы > 3 лет. Кандидат указывает должность, компанию,учебное заведение: медицинские университеты или институты. \n\
            Высшее медицинское образование - это медицинские университеты и институты. Среднее медицинское образование - это медицинский колледж, школы обучающие косметологов. Стаж работы - это опыт работы в сфере косметологии. \n\
            Обрати внимание на класс 3 и 4 прежде чем принять решение.\n\ Не обращай внимание на образование, оно не влияет на метку класса \n\
            Три примера для  каждого класса:\n\
                \"text\": {df['info'][df['Оценка'] == 0].sample(n=3).to_list()}, \"label\": 0,\n\
                \"text\": {df['info'][df['Оценка'] == 1].sample(n=3).to_list()}, \"label\": 1,\n\
                \"text\": {df['info'][df['Оценка'] == 2].sample(n=3).to_list()}, \"label\": 2,\n\
                \"text\": {df['info'][df['Оценка'] == 3].sample(n=3).to_list()}, \"label\": 3,\n\
                \"text\": {df['info'][df['Оценка'] == 4].sample(n=3).to_list()}, \"label\": 4,\n\
                \"text\": {df['info'][df['Оценка'] == 5].sample(n=3).to_list()}, \"label\": 5,\n\
                \"text\": {df['info'][df['Оценка'] == 6].sample(n=3).to_list()}, \"label\": 6,\n\
                \"text\": {df['info'][df['Оценка'] == 7].sample(n=3).to_list()}, \"label\": 7,\n\
                \"text\": {df['info'][df['Оценка'] == 8].sample(n=3).to_list()}, \"label\": 8\n\
            Угадай класс данного резюме: {data['info'][i]}. Дай ответ в формате json с двумя ключами 'text' - полученный текст и prediction - угаданная метка класса" }]
  answer =  deep_seekgpt(messages, temperature = 0.8)
  answer1 = json.loads(answer)
  result4[i] = answer1['prediction']

In [ ]:
result4 = pd.DataFrame.from_dict(result4, orient='index', columns=['prediction4'])

#Окончательное решение

In [ ]:
majority_vote = Counter(result).most_common(1)[0][0]

In [ ]:
majority_vote = {}

In [ ]:
for i,v in result.iterrows():
  majority_vote[i] = Counter(v).most_common(1)[0][0]


In [ ]:
y_pred = pd.DataFrame.from_dict(majority_vote, orient='index')
y_pred = np.array(y_pred)

In [ ]:
print("Accuracy :", accuracy_score(y, y_pred))
print("F1 score:",f1_score(y, y_pred, average='weighted'))

Accuracy : 0.70509977827051
F1 score: 0.6940426620895117


In [ ]:
# Сначала получим список всех классов
classes = np.unique(y)

# 1) F1-score по классам
f1_per_class = f1_score(y, y_pred, labels=classes, average=None, zero_division=0)

# 2) «Точность по классу» как доля правильно предсказанных среди всех примеров этого класса
#    (этот показатель в мультиклассе эквивалентен recall для данного класса)
accuracy_per_class = np.array([
    np.mean(y_pred[y == cls] == cls)
    for cls in classes
])

# 3) Собираем всё в DataFrame
df = pd.DataFrame({
    'class': classes.astype(str),
    'accuracy': accuracy_per_class,
    'f1_score': f1_per_class
}).set_index('class')

print(df)

       accuracy  f1_score
class                    
0      0.882353  0.857143
1      0.803922  0.766355
2      0.686275  0.777778
3      0.352941  0.473684
4      0.348837  0.491803
5      0.666667  0.747253
6      0.960784  0.777778
7      0.862745  0.637681
8      0.725490  0.685185
